In [13]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from datetime import datetime
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from sqlalchemy import create_engine

In [14]:
# Set up the connection to the database
engine = create_engine('sqlite:///mydatabase.db')

def print_time_and_store_in_db(**kwargs):
    # Get the current time and format it as a string
    now = datetime.now()
    formatted_time = now.strftime('%Y.%m.%d %H:%M:%S')
    
    # Print the current time
    print(f'Current Time: {formatted_time}')
    
    # Store the current time in the database
    job_number = kwargs['ti'].job_id.split('_')[-1]
    query = f"INSERT INTO mytable (job_number, job_output, job_executed_at) VALUES ({job_number}, '{formatted_time}', '{formatted_time}')"
    engine.execute(query)

In [16]:
# Define default_args for your DAG
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2022, 1, 1)
}

# Instantiate a DAG object with an ID of 'my_dag'
dag = DAG(
    dag_id='my_dag',
    default_args=default_args,
    schedule_interval='@hourly',  # Run once every hour
)

# Create a PythonOperator task that calls your function
task = PythonOperator(
    task_id='print_time_and_store_in_db',
    python_callable=print_time_and_store_in_db,
    provide_context=True,
    dag=dag,
)